
**L'objectif principal** de cette partie est de développer une fonction capable d'automatiser la labélisation du contenu des rapports SFCR en catégorisant chaque élément textuel en trois classes distinctes :

Inutile : Contenu non pertinent comme les bas de page, les hauts de page, ou les tableaux.
Paragraphe : Sections de texte informatif appartenant au corps principal du rapport.
Titre : Grands titres ou sous-titres délimitant les différentes parties du rapport.



In [2]:
import os
import pandas as pd
import numpy as np
import json
import os

**Extraction et structuration des informations**

La fonction produce_brut(assureur) extrait et structure des informations textuelles et de mise en page à partir d'un fichier JSON contenant les données OCR.

In [10]:
def produce_brut(assureur):
    with open('/content/{}.json'.format(assureur), 'r',encoding="utf8") as f:
        data = json.load(f)
    pages_content=data["responses"]
    num_page=0
    df=[]
    for page in pages_content:
        num_page+=1
        if "fullTextAnnotation" not in page:
            continue
        p=page["fullTextAnnotation"]["pages"]
        for e in p:
            blocks=e["blocks"]
            page_features=[]
            for block in blocks:
                for para in block["paragraphs"]:
                    # collect text
                    text = ""
                    for word in para["words"]:
                        #print("-----")
                        #print(word)
                        for symbol in word["symbols"]:
                            if symbol["confidence"]>=0.8:
                                text += symbol["text"]
                        text+=" "
                    # extract bounding box features
                    x_list = []
                    y_list = []
                    for v in para["boundingBox"]["normalizedVertices"]:
                        x_list.append(v["x"])
                        y_list.append(v["y"])
                    f = {}
                    f["num_page"]=num_page
                    f["text"] = text
                    f["width"] = max(x_list) - min(x_list)
                    f["height"] = max(y_list) - min(y_list)
                    f["area"] = f["width"] * f["height"]
                    f["chars"] = len(text)
                    f["char_size"] = f["area"] / f["chars"] if f["chars"] > 0 else 0
                    f["pos_x"] = (f["width"] / 2.0) + min(x_list)
                    f["pos_y"] = (f["height"] / 2.0) + min(y_list)
                    f["aspect"] = f["width"] / f["height"] if f["height"] > 0 else 0
                    f["layout"] = "h" if f["aspect"] > 1 else "v"
                    f["x0"]=x_list[0]
                    f["x1"]=x_list[1]
                    f["y0"]=y_list[0]
                    f["y1"]=y_list[1]
                    page_features.append(f)
            df=df+page_features
    df=pd.DataFrame(df)
    df["assureur"]=assureur
    return df

In [29]:
# Exemple d'appel de la fonction et sauvegarde en CSV
assureur = "allianz-1-to-94"  # Remplacez par le nom du fichier JSON sans l'extension
df_result = produce_brut(assureur)
df_result

,num_page,text,width,height,area,chars,char_size,pos_x,pos_y,aspect,layout,x0,x1,y0,y1,assureur
0,1,Allianz,0.250420,0.030879,0.007733,9,0.000859,0.180672,0.112827,8.109758,h,0.055462,0.305882,0.097387,0.098575,allianz-1-to-94
1,1,ALLIANZ VIE,0.117647,0.010689,0.001258,12,0.000105,0.114286,0.215558,11.006553,h,0.055462,0.173109,0.210214,0.210214,allianz-1-to-94
2,1,Rapport sur la solvabilité,0.663866,0.041568,0.027595,27,0.001022,0.392437,0.271378,15.970699,h,0.060504,0.724370,0.256532,0.250594,allianz-1-to-94
3,1,Exercice 2022,0.121008,0.010689,0.001293,14,0.000092,0.115966,0.437648,11.320995,h,0.055462,0.176471,0.432304,0.432304,allianz-1-to-94
4,1,et la situation financière,0.638655,0.033254,0.021238,27,0.000787,0.613445,0.321853,19.205268,h,0.294118,0.932773,0.306413,0.305226,allianz-1-to-94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4166,94,Entreprise régie par le Code des assurances So...,0.322689,0.038005,0.012264,107,0.000115,0.243697,0.874109,8.490757,h,0.082353,0.405042,0.856294,0.855107,allianz-1-to-94
4167,94,1 cours Michelet - CS 30051 92076 Paris La Déf...,0.196639,0.022565,0.004437,57,0.000078,0.180672,0.906770,8.714202,h,0.082353,0.278992,0.895487,0.895487,allianz-1-to-94
4168,94,340 234 962 R.C.S. Nanterre,0.183193,0.007126,0.001305,28,0.000047,0.173950,0.927553,25.708092,h,0.082353,0.265546,0.923991,0.923991,allianz-1-to-94
4169,94,,0.023529,0.014252,0.000335,1,0.000335,0.850420,0.923991,1.650981,h,0.838655,0.862185,0.916865,0.916865,allianz-1-to-94


**Traitrement et labelisation des données**

La fonction label_content(df, thresholds=None) attribue une étiquette (ou "Label") à chaque ligne d'un DataFrame, en classifiant le contenu en trois catégories :

1.  **Inutile :** Contenu jugé non pertinent, comme les en-têtes, pieds de page ou contenu de tableau.
2.**Paragraphe :** Contenu jugé pertinent pour le corps du texte.
1. **Titre :** Sections marquées par de courts textes, généralement des titres ou sous-titres.

In [30]:
def label_content(df, thresholds=None):
    """
    Ajoute une colonne 'Label' au DataFrame pour classifier le contenu en 'Inutile', 'Paragraphe', ou 'Titre'.

    Paramètres :
    - df : DataFrame produit par la fonction produce_brut().
    - thresholds : Dictionnaire contenant des seuils ajustables pour le classement.

    Retour :
    - df : DataFrame avec une colonne supplémentaire 'Label'.
    """

    # Vérifier si df est un DataFrame
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Le paramètre 'df' doit être un DataFrame.")

    # Vérifier si les colonnes nécessaires sont présentes
    required_columns = {'pos_y', 'height', 'char_size', 'chars'}
    if not required_columns.issubset(df.columns):
        raise ValueError(f"Le DataFrame doit contenir les colonnes suivantes : {required_columns}")

    # Seuils par défaut
    default_thresholds = {
        "header_footer_y": 0.1,       # Seuil pour détecter le haut/bas de page
        "char_count_title": 50,       # Nombre maximum de caractères pour un titre
        "char_count_paragraph": 51,   # Nombre minimum de caractères pour un paragraphe
        "char_size_table": 0.0005,    # Taille des caractères indicative pour le contenu de tableau
        "height_table": 0.01          # Hauteur indicative pour le contenu de tableau
    }

    # Utiliser les seuils fournis par l'utilisateur ou les seuils par défaut
    thresholds = {**default_thresholds, **(thresholds or {})}

    def classify(row):
        """Classifie une ligne en fonction des seuils."""
        # Détection des en-têtes et pieds de page (positions proches des bords verticaux)
        if row['pos_y'] < thresholds["header_footer_y"] or row['pos_y'] > (1 - thresholds["header_footer_y"]):
            return "Inutile"

        # Détection du contenu de tableau (petite hauteur et taille de caractère faible)
        if row['height'] < thresholds["height_table"] and row['char_size'] < thresholds["char_size_table"]:
            return "Inutile"

        # Détection des titres (peu de caractères)
        if row['chars'] <= thresholds["char_count_title"]:
            return "Titre"

        # Détection des paragraphes (nombre de caractères élevé)
        if row['chars'] > thresholds["char_count_paragraph"]:
            return "Paragraphe"

        # Par défaut, marquer comme "Inutile"
        return "Inutile"

    # Appliquer la fonction de classification à chaque ligne du DataFrame
    df['Label'] = df.apply(classify, axis=1)
    return df

In [31]:
if __name__ == "__main__":
    try:
        # Charger les données brutes
        df = produce_brut(assureur)

        # Appliquer les labels
        df_labeled = label_content(df)

        # Afficher un aperçu du DataFrame labellisé
        print(df_labeled[['num_page', 'text', 'Label']])

        # Sauvegarder le DataFrame en CSV
        output_dir = "output"
        os.makedirs(output_dir, exist_ok=True)  # Crée le répertoire 'output' s'il n'existe pas
        output_file = os.path.join(output_dir, f"{assureur}_labeled.csv")
        df_labeled.to_csv(output_file, index=False, encoding="utf-8-sig")

        print(f"Le fichier CSV a été sauvegardé sous : {output_file}")

    except FileNotFoundError as fnf_error:
        print(fnf_error)
    except ValueError as val_error:
        print(f"Erreur de validation des données : {val_error}")
    except Exception as e:
        print(f"Erreur imprévue : {e}")

      num_page                                               text       Label
0            1                                          Allianz         Titre
1            1                                       ALLIANZ VIE        Titre
2            1                        Rapport sur la solvabilité        Titre
3            1                                     Exercice 2022        Titre
4            1                        et la situation financière        Titre
...        ...                                                ...         ...
4166        94  Entreprise régie par le Code des assurances So...  Paragraphe
4167        94  1 cours Michelet - CS 30051 92076 Paris La Déf...     Inutile
4168        94                       340 234 962 R.C.S. Nanterre      Inutile
4169        94                                                        Inutile
4170        94  Réalisation interne Allianz  Crédits photos  i...  Paragraphe

[4171 rows x 3 columns]
Le fichier CSV a été sauvegardé sous : 

**Filtrage des données et génération de ficher texte**

Ce code filtre les données inutiles et génère un fichier .txt à partir des données traitées.

1.   Chargement et traitement des données
2.   Filtrage du contenu pertinent
1.   Génération du .txt






In [32]:
if __name__ == "__main__":
    # Exemple avec un fichier 'assureurX.json' situé dans le dossier 'data/ocr/'
    try:
        df = produce_brut(assureur)
        df_labeled = label_content(df)

        # Filtrer pour ne garder que les titres et les paragraphes
        df_filtered = df_labeled[df_labeled['Label'].isin(['Titre', 'Paragraphe'])]

        # Afficher un aperçu du DataFrame filtré
        print(df_filtered[['num_page', 'text', 'Label']])

        # Sauvegarder le contenu filtré dans un fichier texte
        output_dir = "output"
        os.makedirs(output_dir, exist_ok=True)  # Crée le répertoire 'output' s'il n'existe pas
        txt_file = os.path.join(output_dir, f"{assureur}_report.txt")

        # Fonction pour générer le fichier texte
        def generate_text(dataframe, filename):
            with open(filename, 'w', encoding='utf-8') as f:
                current_page = None
                for _, row in dataframe.iterrows():
                    # Ajouter un saut de ligne pour indiquer un changement de page
                    if current_page is None or row['num_page'] != current_page:
                        if current_page is not None:
                            f.write("\n" + "="*50 + "\n")  # Séparateur pour une nouvelle page
                        current_page = row['num_page']
                        f.write(f"\nPage {current_page}\n")

                    # Ajouter le texte en fonction du label
                    if row['Label'] == 'Titre':
                        f.write(f"\n{row['text']}\n")
                    elif row['Label'] == 'Paragraphe':
                        f.write(f"{row['text']}\n")

                    # Ajouter un espacement après chaque élément
                    f.write("\n")

            print(f"Le fichier texte a été sauvegardé sous : {filename}")

        # Générer le fichier texte avec le contenu filtré
        generate_text(df_filtered, txt_file)

    except FileNotFoundError:
        print(f"Le fichier data/ocr/{assureur}.json est introuvable.")
    except Exception as e:
        print(f"Erreur : {e}")


      num_page                                               text       Label
0            1                                          Allianz         Titre
1            1                                       ALLIANZ VIE        Titre
2            1                        Rapport sur la solvabilité        Titre
3            1                                     Exercice 2022        Titre
4            1                        et la situation financière        Titre
...        ...                                                ...         ...
4158        93  Figure 10  Structure des risques en Modèle Int...  Paragraphe
4162        93  Figure 11  Structure des risques en Formule St...  Paragraphe
4164        94                                           Allianz        Titre
4166        94  Entreprise régie par le Code des assurances So...  Paragraphe
4170        94  Réalisation interne Allianz  Crédits photos  i...  Paragraphe

[1327 rows x 3 columns]
Le fichier texte a été sauvegardé sous 